<a href="https://colab.research.google.com/github/luisfocosta/ai_playground/blob/main/L3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 3: Building Agents with memory

## Preparation

<div style="background-color:#fff6ff; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
<p> 💻 &nbsp; <b>Access <code>requirements.txt</code> and <code>helper.py</code> files:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>.

<p> ⬇ &nbsp; <b>Download Notebooks:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Download as"</em> and select <em>"Notebook (.ipynb)"</em>.</p>

<p> 📒 &nbsp; For more help, please see the <em>"Appendix – Tips, Help, and Download"</em> Lesson.</p>
</div>

<p style="background-color:#f7fff8; padding:15px; border-width:3px; border-color:#e0f0e0; border-style:solid; border-radius:6px"> 🚨
&nbsp; <b>Different Run Results:</b> The output generated by AI models can vary with each execution due to their dynamic, probabilistic nature. Your results may differ from those shown in the video.</p>

Letta agents persist information over time and restarts by saving data to a database. These lessons do not require past information. To enable a clean restart, the database is cleared before starting the lesson.

In [ ]:
!rm  -f ~/.letta/sqlite.db

## Section 0: Setup a client

In [ ]:
import os
print(os.getcwd())
for filename in os.listdir():
    print(filename)

/content
.config
sample_data


In [ ]:
from helper import nb_print

ImportError: cannot import name 'nb_print' from 'helper' (unknown location)

In [ ]:
from letta import create_client

client = create_client()

Saved Config:  /home/jovyan/.letta/config
📖 Letta configuration file updated!
🧠 model	-> gpt-4
🖥️  endpoint	-> http://jupyter-api-proxy.internal.dlai/rev-proxy/letta
Saved Config:  /home/jovyan/.letta/config
Saved Config:  /home/jovyan/.letta/config


In [ ]:
from letta.schemas.llm_config import LLMConfig

client.set_default_llm_config(LLMConfig.default_config("gpt-4o-mini"))

## Section 1: Creating a simple agent with memory

### Creating an agent

In [ ]:
agent_name = "simple_agent"

In [ ]:
# this is not in the video. It deletes the agent if you are running this a 2nd time.
if client.get_agent_id(agent_name):
    client.delete_agent(client.get_agent_id(agent_name))

In [ ]:
from letta.schemas.memory import ChatMemory

agent_state = client.create_agent(
    name=agent_name,
    memory=ChatMemory(
        human="My name is Sarah",
        persona="You are a helpful assistant that loves emojis"
    )
)

In [ ]:
response = client.send_message(
    agent_id=agent_state.id,
    message="hello!",
    role="user"
)

Letta.letta.server.server - ERROR - Error in server._step: 500 Server Error: Internal Server Error for url: http://jupyter-api-proxy.internal.dlai/rev-proxy/letta/chat/completions


Traceback (most recent call last):
  File "/usr/local/lib/python3.11/site-packages/letta/server/server.py", line 323, in _step
    step_response = letta_agent.step(
                    ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/site-packages/letta/agent.py", line 897, in step
    raise e
  File "/usr/local/lib/python3.11/site-packages/letta/agent.py", line 804, in step
    response = self._get_ai_reply(
               ^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/site-packages/letta/agent.py", line 495, in _get_ai_reply
    raise e
  File "/usr/local/lib/python3.11/site-packages/letta/agent.py", line 464, in _get_ai_reply
    response = create(
               ^^^^^^^
  File "/usr/local/lib/python3.11/site-packages/letta/llm_api/llm_api_tools.py", line 197, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/site-packages/letta/llm_api/llm_api_tools.py", line 329, in create
    response = openai_chat_completions

None


HTTPError: 500 Server Error: Internal Server Error for url: http://jupyter-api-proxy.internal.dlai/rev-proxy/letta/chat/completions

In [ ]:
response.usage

In [ ]:
nb_print(response.messages)

### Understanding agent state

In [ ]:
print(agent_state.system)

In [ ]:
agent_state.tools

### Viewing an agent's memory

In [ ]:
agent_state.memory

In [ ]:
client.get_archival_memory_summary(agent_state.id)

In [ ]:
client.get_recall_memory_summary(agent_state.id)

In [ ]:
client.get_messages(agent_state.id)[0]

## Section 2: Understanding core memory

### Memories about the human

In [ ]:
response = client.send_message(
    agent_id=agent_state.id,
    message = "My name is actually Bob",
    role = "user"
)
nb_print(response.messages)

### Memories about the agent

In [ ]:
response = client.send_message(
    agent_id=agent_state.id,
    message = "In the future, never use emojis to communicate",
    role = "user"
)
nb_print(response.messages)

In [ ]:
client.get_core_memory(agent_state.id).get_block('persona')

## Section 3: Understanding archival memory

In [ ]:
client.get_archival_memory(agent_state.id)

In [ ]:
response = client.send_message(
    agent_id=agent_state.id,
    message = "Save the information that 'bob loves cats' to archival",
    role = "user"
)
nb_print(response.messages)

In [ ]:
client.get_archival_memory(agent_state.id)[0].text

In [ ]:
passage = client.insert_archival_memory(
    agent_state.id,
    "Bob loves Boston Terriers"
)

In [ ]:
passage

In [ ]:
response = client.send_message(
    agent_id=agent_state.id,
    role="user",
    message="What animals do I like? Search archival."
)
nb_print(response.messages)